## ECE 269 Mini Project 1

In [27]:
!pip install numpy
import numpy as np
import random
import helper_func as F

## 2. Experimental Setup

1. Model: $$ y = Ax +n$$
$y \in R^M$ - measurements
- $A \in R^{M\times N}$ - measurement matrix
- $x \in R^N$ - sparse signal to solve for with random support cardinality $s$
- $n \in R^N$ - additive noise, st

In [28]:
def genAmatrix (m,n):
    # mxn normalized matrix sampled from standard normal distribution 
    A = []
    for i in range (n):
        column = np.random.randn(m)
        columnNorm = F.l2Norm(column)
        column = column/columnNorm      # normalize columns
        A.append(column)
    A = np.array(A).T
    return A

def genSparseX(n,s):
    # nx1 sparse vector x with random support cardinality s
    # s ~ [1,N], x_i = ~ [-10, -1] U [1, 10]
    s_idx = np.random.choice(n, size=s, replace=False)  # indexes of sparce coeff
    x = np.zeros(n)
    mag = np.random.rand(s)*9+1    # [1, 10]
    sign = np.random.choice([-1,1], size=s)

    x[s_idx] = sign * mag   # [-10, -1] U [1, 10]
    return x, s_idx 


In [29]:
# TEST
test_A = genAmatrix(5,4)
print("A: \n", test_A)
n=100
s=50
test_x, s_idx= genSparseX(n,s)
import numpy as np


# test_gen_sparse_X()
# print("x: ", test_x)
print("# s", np.count_nonzero(test_x), s, s_idx.shape)
print("# n ", test_x.shape, n)



A: 
 [[-0.5031216  -0.1225766  -0.72237828  0.29727831]
 [-0.32250613  0.48077446  0.03684174 -0.33520947]
 [ 0.14226545  0.3654372   0.46078792  0.29557787]
 [-0.42778811 -0.77599848  0.51354895  0.8380595 ]
 [ 0.66303569 -0.13458419  0.02746422  0.09772524]]
# s 50 50 (50,)
# n  (100,) 100


# Orthogonal Matching Pursuit (OMP) 
$$
\lambda_k = \arg\max_{\omega} \left| \langle r_{k-1}, \phi_{\omega} \rangle \right|
$$

$\phi_{\omega}$ is the $\omega$ th term of A

In [30]:
# OMP Algorithm
# takes measurements y, measurement
# ouputs signal x with s atom indices

def OMP(A, y, stop, tolerance = 1e-6):                  # max iterations should be n
    m, n = A.shape
    r = y                                               # r_0 = s
    lambdas = []                                        # support
    x_k = np.zeros(n)

    i = 0
    while(i < stop):                                    # stop condition: max iterations
        correlation = np.abs(A.T @ r)                   # correlation of each atom with current residual
        lambda_k = np.argmax(correlation)               # ith best atom (max correlation)
        print("\n",i, "th best atom: ", lambda_k)
        lambdas.append(lambda_k)
        
        A_sel = A[:, lambdas]                           # basis of atoms matrix
        ls = F.leastSquare(A_sel, y)
        x_k[lambdas] = ls

        r = y - (A_sel @ ls)                            # calc residual

        if (F.l2Norm(r) < tolerance):                   # error reached below tolerance
            break

        i += 1

    return x_k, lambdas


# 3. Noiseless case: (n = 0)
Find the probability of Exact Support Recover $P_{ESR}$

We use the perfomance metric: where $\hat{x}$ is the estimate of x obtained from OMP, and the normalized error is:
$$
\varepsilon^{(t)}
=
\frac{
\| x^{(t)} - \hat{x}^{(t)} \|_2
}{
\| x^{(t)} \|_2
}
$$

In [34]:
def genAll(m, n, s):
    A = genAmatrix(m, n)
    x, s_idx = genSparseX(n, s)
    y = A @ x
    return A, x, s_idx, y

def runNoiseless(m, n, s_max, times = 2000, tolerance=1e-6):
    # sweep through all size s from 1 to n
    count = 0
    error = 0
    for i in range(times):
        s = int(np.random.rand() * (s_max-1)) + 1       # bias of 1 s~(1, s_max)
        A, x, s_idx, y = genAll(m, n, s)
        x_k, lambdas = OMP(A, y, s, tolerance)
        if(set(lambdas) == set(s_idx)):
            count += 1 

        error = error + F.normalError(x, x_k)
    
    p_ESR = count / times                               # probability of exact support recovery
    error_mean = error / times                          # average normalized error

    return p_ESR, error_mean



In [35]:
N_list = [20, 50, 100]
